In [1]:
suppressWarnings(suppressMessages(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


In [2]:
fdiry = file.path(FD_RES, "source", "CRISPRi_FlowFISH")
fname = "K562_HCRFF_20genes_TSS_TES_coords.out"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, show_col_types = FALSE)

dat_gene_info = dat
print(dim(dat))
head(dat)

[1] 20  5


gene,chr,strand,TSS,TES
<chr>,<chr>,<chr>,<dbl>,<dbl>
CAPRIN1,chr11,+,34051730,34102610
CAT,chr11,+,34438933,34472060
CD164,chr6,-,109366513,109382467
ERP29,chr12,+,112013347,112023220
FADS1,chr11,-,61799628,61817003
FADS2,chr11,+,61816202,61867354


In [3]:
genes = dat_gene_info$gene
print(length(genes))
print(genes)

[1] 20
 [1] "CAPRIN1" "CAT"     "CD164"   "ERP29"   "FADS1"   "FADS2"   "FADS3"  
 [8] "FEN1"    "GATA1"   "HBE1"    "HBG1"    "HBG2"    "HBS1L"   "HDAC6"  
[15] "LMO2"    "MEF2C"   "MYB"     "MYC"     "NMU"     "PVT1"   


In [4]:
fdiry = file.path(FD_RES, "source", "CRISPRi_FlowFISH", "track_bedgraph")

lst = lapply(genes, function(gene){
    
    ###
    fname = paste(gene, "HS_exp_r1.tsv.gz", sep="_")
    fpath = file.path(fdiry, fname)
    
    ### read data
    cnames = c("Chrom", "Start", "End", "Name", "SeqCounts", "Strand", "Guide_ID", 
           "Chrom_TSS", "Start_TSS", "End_TSS", "Strand_Gene", 
           "Gene_Symbol", 
           "Gene_ENS", 
           "Guide_SpacerSeq",
           "Guide_Seq",
           "Guide_Type",
           "Notes"
          )
    ctypes = cols(
        "Chrom" = col_character(),
        "Start" = col_integer(),
        "End"   = col_integer()
    )
    dat = read_tsv(fpath, col_names = cnames, col_types = ctypes, show_col_types = FALSE)
    return(dat)
})
names(lst) = genes

lst_crispr_info = lst
for (idx in names(lst)){
    dat = lst[[idx]]
    txt = format(idx, width = 7, justify = "left")
    cat(txt, dim(dat), "\n")
}

CAPRIN1 60000 17 
CAT     60000 17 
CD164   60001 17 
ERP29   60000 17 
FADS1   10000 17 
FADS2   10000 17 
FADS3   10000 17 
FEN1    10000 17 
GATA1   15335 17 
HBE1    21170 17 
HBG1    21170 17 
HBG2    21170 17 
HBS1L   21170 17 
HDAC6   15335 17 
LMO2    60000 17 
MEF2C   60000 17 
MYB     21170 17 
MYC     51130 17 
NMU     45500 17 
PVT1    51130 17 


In [5]:
dat = lst[[1]]
head(dat)

Chrom,Start,End,Name,SeqCounts,Strand,Guide_ID,Chrom_TSS,Start_TSS,End_TSS,Strand_Gene,Gene_Symbol,Gene_ENS,Guide_SpacerSeq,Guide_Seq,Guide_Type,Notes
<chr>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
chr11,61748850,61748853,NA|chr11:61748831-61748850:+,121,+,chr11:61748831-61748850:+,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,CTTTACCTCCACTGTAAGGC,GCTTTACCTCCACTGTAAGGC,negative_control,STT
chr11,61748851,61748854,NA|chr11:61748832-61748851:+,153,+,chr11:61748832-61748851:+,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,TTTACCTCCACTGTAAGGCA,GTTTACCTCCACTGTAAGGCA,negative_control,STT
chr11,61748971,61748974,NA|chr11:61748975-61748994:-,172,-,chr11:61748975-61748994:-,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,CTACCGAGCCACAGTGAGTG,GCTACCGAGCCACAGTGAGTG,negative_control,STT
chr11,61749002,61749005,NA|chr11:61748983-61749002:+,3,+,chr11:61748983-61749002:+,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,GTGGCTCGGTAGAGAAGCCA,GTGGCTCGGTAGAGAAGCCA,negative_control,STT
chr11,61749033,61749036,NA|chr11:61749014-61749033:+,246,+,chr11:61749014-61749033:+,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,GAGGACTCATCTCCATTGAT,GAGGACTCATCTCCATTGAT,negative_control,STT
chr11,61749038,61749041,NA|chr11:61749042-61749061:-,851,-,chr11:61749042-61749061:-,chr11,34051731,34051732,+,CAPRIN1,ENSG00000135387,ACTTGCATACAAGGACCTAG,GACTTGCATACAAGGACCTAG,negative_control,STT


In [7]:
lst = lst_crispr_info
lst = lapply(lst, function(dat){
    dat = dat %>% 
       dplyr::filter(Guide_Type == "targeting") %>%
       dplyr::select(Chrom, Start, End)
    return(dat)
})

In [10]:
dat = bind_rows(lst) %>% distinct
dim(dat)

[1] 344689      3